In [1]:
import brightway2 as bw
import functools

## Logic of import
- a database is imported from an excel file using the bw2 default functions.
- the activities in this database are matched with the existing databases (user provided) in the project.
- if there are any unlinked activities (From ecoinvent), they are stored as a separate **list of dict**.
-   then the whole table of linked and unlinked activities are stored as an excel sheet.
- The unlinked activities are dropped, the database is written
- The unlinked activities are matched against the ecoinvent database and the unique (based on the reference product, name and location) ecoinvent counterparts are identified.
-   These unique activities are written to the newly imported database and formulas, if any, are inherited from the **list of dict**. Thus all the originally unlinked activities are copied back into the new database
- Now, activity browser can be opened and these originally unlinked activities can be reassigned by dragging and dropping.

In [2]:
from bw2io import *
from bw2io.strategies import *

In [3]:
bw.projects.set_current('alicia')

In [4]:
import select_file_v2 as select
database_path, database_name = select.file_selection()
# for checking the paths
print(database_path)
print(database_name)

D:\Alicia\Work\Git\brightway\databases\ATA-72_Engine_fan_prop.xlsx
ATA-72_Engine_fan_prop.xlsx


In [5]:
# Create an ExcelImporter object
ei = ExcelImporter(database_path)

Extracted 1 worksheets in 0.01 seconds


In [6]:
# apply the statistics and check for data statistics
ei.apply_strategies()
ei.statistics()

Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applying strategy: convert_activity_parameters_to_list
Applied 16 strategies in 11.83 seconds
7 datasets
22 exchanges
10 unlinked exchanges
  Type technosphere: 6 unique unlinked exchanges


(7, 22, 10)

In [7]:
ei.match_database(fields=('name')) #database self matching

import db_dependencies as DB
for i in range (DB.db_array_length):
    ei.match_database(db_name=DB.db_names[i], fields=('name', 'unit', 'location'))
#ei.match_database(None, ignore_categories=True)
ei.statistics()

Applying strategy: link_iterable_by_fields
Couldn't apply strategy link_iterable_by_fields:
	Object in source database can't be uniquely linked to target database.
Problematic dataset is:
{'a': '(missing)',
 'e': '(missing)',
 'filename': '(missing)',
 'm': '(missing)',
 'n': '(missing)'}
Possible targets include (at least one not shown):
[{'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'},
 {'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'},
 {'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'},
 {'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'},
 {'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'},
 {'a': '(missing)',
  'e': '(missing)',
  'filename': '(missing)',
  'm': '(missing)',
  'n': '(missing)'}]
Applying st

(7, 22, 0)

In [11]:
# To "neatly" print the entire imported data within ei. ONLY use when requried
import pprint
pprint.pprint(ei.data)


[{'code': 'a54d4bf5d142483190d6e3a496a210ad',
  'database': 'ATA-72_Engine_fan_prop',
  'exchanges': [{'amount': 1,
                 'database': 'ATA-72_Engine_fan_prop',
                 'input': ('ATA-72_Engine_fan_prop',
                           'a54d4bf5d142483190d6e3a496a210ad'),
                 'location': 'GLO',
                 'name': 'ATA-72_Engine_fan_prop_Engine Systems',
                 'output': ('ATA-72_Engine_fan_prop',
                            'a54d4bf5d142483190d6e3a496a210ad'),
                 'reference product': 'Engine Systems',
                 'type': 'production',
                 'unit': 'unit'},
                {'amount': 274.53,
                 'database': 'materials',
                 'formula': 'engineSystems_mass',
                 'input': ('materials', '31f1a80380444c41ad2b1b1cceb9d5d5'),
                 'location': 'GLO',
                 'name': 'Al 2024',
                 'original_amount': 1,
                 'output': ('ATA-72_Engine_fan_

In [ ]:
# if there are any unlinked exchanges, the databases of these exchanges, and the unlinked exchanges are stored in their own arrays
unlinked_databases = []
unlinked_exchanges = []
for exc in ei.unlinked:
    if exc["database"] not in unlinked_databases:
        unlinked_databases.append(exc["database"])
    unlinked_exchanges.append(exc)
# the unlinked exchanges are stored as a list, where each element is a dict
print ("The unlinked database/s:\n",unlinked_databases)
print("The unlinked exchanges are:\n",unlinked_exchanges)

In [ ]:
print(unlinked_exchanges[1]["name"])

In [ ]:
#write an excel file with all the matched activities so far.copy the excel file that was written in the previous line to your desired location for further reference.
ei.write_excel(only_unlinked=False)
ei.add_unlinked_activities()
ei.statistics()

# ei.drop_unlinked(i_am_reckless=True)

In [ ]:
#uncomment the line below to enable the Nuclear option
ei.drop_unlinked(i_am_reckless=True)

In [8]:
ei.statistics()

7 datasets
22 exchanges
0 unlinked exchanges
  


(7, 22, 0)

In [9]:
# will write to the database only if there are no unlinked exchanges
#writes the database into the bw2 as the same name of the excel sheet
ei.write_database()

Writing activities to SQLite3 database:
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 05/15/2023 14:31:10
  Finished: 05/15/2023 14:31:10
  Total time elapsed: 00:00:00
  CPU %: 0.00
  Memory %: 1.44
Created database: ATA-72_Engine_fan_prop


**if unlinked (with ecoinvent) activities have been dropped**, then the code block below is used for searching and finding similar activities from the ecoinvent database.

**Note**: There is potentially one another method to do this. That is to search the unlinked_exchanges dict, and then add each activity by itself to the new database. However, this will result in loss of connection and attribution with ecoinvent database. May be, this method can be attempted in future.

In [ ]:
print(unlinked_exchanges)
#print(unlinked_exchanges[0]["formula"])

In [ ]:
import bw2data as bd

#select the source ecoinvent database and the target database with missing activities
ecoinvent_db = bd.Database('cutoff391')
target_db = bd.Database('aircraftlca2')
#iterate through each unlinked exchanges stored in the unlinked_exchanges list
for exchange in unlinked_exchanges:
    #print (exchange["name"])

    #matches activity in unlinked_exchanges to those in th ecoinvent_db by name, location and reference product
    matches_activity = [act for act in ecoinvent_db if act["name"] == exchange["name"] and act["location"] == exchange["location"] and act["reference product"] == exchange["reference product"]]
    #matches_activity = bd.Database(database_name).search(exchange["name"])
    for match in matches_activity:
        #saves the activity code from the matched activity. This activity code is later used for copying the activity from ecoinvent db to target database
        activity_code = match["code"]
    #source_activity is the activity to be copies from ecoinvent
    source_activity = ecoinvent_db.get(activity_code)
    print("The activity from ecoinvent is \n", source_activity)

    #copies source activity to a variable called target_activity
    target_activity = source_activity.copy()
    target_activity["database"] = target_db.name

    # if there are any associatted formulas in the unlinked databases it will copied over to the newly added activities in the database. This copy may not work if the parameters are not available.
    if "formula" in exchange.keys():
        print("yes, there is formula")
        target_activity["formula"]=exchange["formula"]

    target_activity.save()
    target_db.process()

In [ ]:
quit()